In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# chen2020mvlidarnet

> Module that implements the models from [MVLidarNet: Real-Time Multi-Class Scene Understanding for Autonomous Driving Using Multiple Views](https://arxiv.org/pdf/2006.05518).

**(UNDER CONSTRUCTION...)**

In [ ]:
#| default_exp chen2020mvlidarnet

In [ ]:
#| export
import torch
from torch.nn import Module, Sequential, Conv2d, BatchNorm2d, ReLU, ConvTranspose2d, MaxPool2d
from torch.nn.init import kaiming_normal_, constant_, zeros_, normal_
from collections import OrderedDict
from lightning import LightningModule
from torchmetrics.classification import Accuracy
from torchmetrics.segmentation import MeanIoU
from torch.nn.modules.module import register_module_forward_hook
import re
import wandb
import numpy as np
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn.functional as F
from PIL import Image

## MVLidarNet Architecture

In [ ]:
#| export
class ConvBNReLU(Sequential):
    "Sequential composition of convolution, batch normalization and ReLU."
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, has_ReLU=True):
        sequence = [
            ('conv', Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)),
            ('bn', BatchNorm2d(out_channels, momentum=0.01))
        ]
        self.nonlinearity = 'linear'
        if has_ReLU:
            self.nonlinearity = 'relu'
            sequence += [('relu', ReLU())]
        super().__init__(OrderedDict(sequence))
        self.init_params()

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('conv\.weight', n):
                kaiming_normal_(p, nonlinearity=self.nonlinearity)
            if re.search('bn\.bias', n):
                constant_(p, 0.1)

In [ ]:
bs, in_c, out_c, h, w = 1, 5, 64, 64, 2048
inp = torch.randn(bs, in_c, h, w)

b = ConvBNReLU(in_c, out_c, 3, 1, 1)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class InceptionV2(Module):
    "InceptionV2 Block from [Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/pdf/1512.00567)."
    def __init__(self, in_channels, out_channels):
        super().__init__()
        out_channels = out_channels//4
        
        # 1x1
        self.b1 = ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        
        # pool -> 1x1
        self.b2 = Sequential(OrderedDict([
            ('pool', MaxPool2d(kernel_size=3 , stride=1 , padding=1)),
            ('cbr', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0))
        ]))
        
        # 1x1 -> 3x3
        self.b3 = Sequential(OrderedDict([
            ('cbr1', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)),
            ('cbr2', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
        ]))
        
        # 1x1 -> 3x3 -> 3x3
        self.b4 = Sequential(OrderedDict([
            ('cbr1', ConvBNReLU(in_channels, out_channels, kernel_size=1, stride=1, padding=0)),
            ('cbr2', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1)),
            ('cbr3', ConvBNReLU(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
        ]))
    
    def forward(self, x):
        branch1 = self.b1(x)
        branch2 = self.b2(x)
        branch3 = self.b3(x)
        branch4 = self.b4(x)
        return torch.cat((branch1, branch2, branch3, branch4), dim=1)

In [ ]:
b = InceptionV2(in_c, out_c)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class InceptionBlock(Sequential):
    "Sequential composition of InceptionV2 modules."
    def __init__(self, in_channels, out_channels, n_modules, has_pool=False):
        modules_in_block = []
        for i in range(n_modules):
            modules_in_block += [(f'incept{i+1}', InceptionV2(in_channels, out_channels))]
            in_channels = out_channels
        if has_pool:
            modules_in_block += [('pool', MaxPool2d(3, 2, 1))] # kernel_size = 3, because of inception v2 paper table 1
        super().__init__(OrderedDict(modules_in_block))

In [ ]:
b = InceptionBlock(in_c, out_c, 2)
outp = b(inp)
assert outp.shape == (bs, out_c, h, w)
print(outp.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class Encoder(Module):
    "MVLidarNet encoder architecture."
    def __init__(self, in_channels=5):
        super().__init__()

        self.trunk1 = ConvBNReLU(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.trunk2 = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.trunk3 = Sequential(
            ConvBNReLU(64, 128, kernel_size=3, stride=1, padding=1),
            MaxPool2d(3, 2, 1) # kernel_size = 3, because of inception v2 paper table 1
        )
        
        self.block1 = InceptionBlock(in_channels=128, out_channels=64, n_modules=2)
        self.block2 = InceptionBlock(in_channels=64, out_channels=64, n_modules=2, has_pool=True)
        self.block3 = InceptionBlock(in_channels=64, out_channels=128, n_modules=3, has_pool=True)
    
    def forward(self, x):
        enc_features = [x]
        
        x = self.trunk1(x)
        x = self.trunk2(x)
        x = self.trunk3(x)
        x = self.block1(x)
        enc_features.append(x)
        
        x = self.block2(x)
        enc_features.append(x)
        
        x = self.block3(x)
        enc_features.append(x)
        
        return enc_features

In [ ]:
enc = Encoder()
outp = enc(inp)
[o.shape for o in outp]

[torch.Size([1, 5, 64, 2048]),
 torch.Size([1, 64, 32, 1024]),
 torch.Size([1, 64, 16, 512]),
 torch.Size([1, 128, 8, 256])]

In [ ]:
#| export
class Decoder(Module):
    "MVLidarNet decoder architecture."
    def __init__(self):
        super().__init__()
        self.up1a = ConvTranspose2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False)
        self.up1c = ConvBNReLU(256+64, 256, kernel_size=1, stride=1, padding=0)
        self.up1d = ConvBNReLU(256, 256, kernel_size=3, stride=1, padding=1)
        
        self.up2a = ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.up2c = ConvBNReLU(128+64, 128, kernel_size=1, stride=1, padding=0)
        self.up2d = ConvBNReLU(128, 128, kernel_size=3, stride=1, padding=1)

        self.up3a = ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, bias=False)
        self.up3b = ConvBNReLU(64, 64, kernel_size=1, stride=1, padding=0)
        self.up3c = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.init_params()

    def init_params(self):
        for n, p in self.named_parameters():
            if re.search('up\da\.weight', n):
                kaiming_normal_(p, nonlinearity='linear')
    
    def forward(self, enc_features):
        x = enc_features[-1]
        
        x = self.up1a(x, output_size=enc_features[-2].size())
        up1b = torch.cat((x, enc_features[-2]), dim=1)
        x = self.up1c(up1b)
        x = self.up1d(x)
        
        x = self.up2a(x, output_size=enc_features[-3].size())
        up2b = torch.cat((x, enc_features[-3]), dim=1)
        x = self.up2c(up2b)
        x = self.up2d(x)
        
        x = self.up3a(x, output_size=enc_features[-4].size())
        x = self.up3b(x)
        x = self.up3c(x)
        
        return x

In [ ]:
dec = Decoder()
fts = dec(outp)
assert fts.shape == (bs, out_c, h, w)
print(fts.shape, f'== ({bs}, {out_c}, {h}, {w})')

torch.Size([1, 64, 64, 2048]) == (1, 64, 64, 2048)


In [ ]:
#| export
class MVLidarNet(Module):
    "MVLidarNet semantic segmentation architecture."
    def __init__(self, in_channels=5, n_classes=7):
        super().__init__()
        self.n_classes = n_classes
        self.input_norm = BatchNorm2d(in_channels, affine=False, momentum=None)
        self.backbone = Sequential(OrderedDict([
            ('enc', Encoder(in_channels)),
            ('dec', Decoder())
        ]))
        
        self.classhead1 = ConvBNReLU(64, 64, kernel_size=3, stride=1, padding=1)
        self.classhead2 = ConvBNReLU(64, n_classes, kernel_size=1, stride=1, padding=0, has_ReLU=False)
    
    def forward(self, x):
        x = self.input_norm(x)
        features = self.backbone(x)
        x = self.classhead1(features)
        prediction = self.classhead2(x)
        return prediction

In [ ]:
n_classes=7
model = MVLidarNet()
logits = model(inp)
assert logits.shape == (bs, n_classes, h, w)
print(logits.shape, f'== ({bs}, {n_classes}, {h}, {w})')

torch.Size([1, 7, 64, 2048]) == (1, 7, 64, 2048)


## LightningModule for standard experiments

In [ ]:
#| exports
def log_activations(logger, step, model, img):
    "Function that uses a Pytorch forward hook to log properties of activations for debugging purposes."
    def debugging_hook(module, inp, out):            
        if hasattr(module, 'name') and 'relu' in module.name:
            acts = out.detach()
            
            min_count = (acts < 1e-1).sum((0, 2, 3))
            shape = acts.shape
            total_count = shape[0]*shape[2]*shape[3]
            rate = min_count/total_count
            logger.log({"max_dead_rate/" + str(module.name): rate.max()}, step=step)
            
            #acts_flat = acts.cpu().view(-1,)
            #acts_hist = np.histogram(acts_flat.log1p(), 100)
            #logger.log({"relu_hist/" + str(module.name): wandb.Histogram(np_histogram=acts_hist)}, step=step)
            
    with register_module_forward_hook(debugging_hook):
        model(img)

In [ ]:
#| export
def log_imgs(pred, label, mask, viz_tfm, logger, stage, step, save_image_locally=True):
    """
    Logs predicted and ground truth images during model training/evaluation, 
    optionally saving them locally.
    """
    def to_numpy(tensor):
        return tensor[0].detach().cpu().numpy()
    
    pred_np = to_numpy(pred).argmax(0)
    label_np = to_numpy(label)
    mask_np = to_numpy(mask)

    altered_pred_np = pred_np.copy()
    # Set matching predictions to 0
    altered_pred_np[altered_pred_np == label_np] = 0
    
    item = {
        "frame": None,
        "label": pred_np,
        "mask": mask_np,
        "weight": None
    }
    pred_img = viz_tfm(item)["label"]

    item["label"] = altered_pred_np
    altered_pred_img = viz_tfm(item)["label"]
    
    item["label"] = label_np
    label_img = viz_tfm(item)["label"]
    
    img_cmp = np.concatenate((label_img, altered_pred_img, pred_img), axis=0)
    
    img_cmp_wandb = wandb.Image(img_cmp)
    logger.log({f"{stage}_examples": img_cmp_wandb}, step=step)

    if save_image_locally:
        img_cmp = img_cmp.astype(np.uint8)
        img_cmp_pil = Image.fromarray(img_cmp)
        save_path = f"../../step_images/{stage}examples_step{step}.png"
        img_cmp_pil.save(save_path)

In [ ]:
#| exports
class SemanticSegmentationTask(LightningModule):
    "Lightning Module to standardize experiments with semantic segmentation tasks."
    def __init__(self, model, loss_fn, viz_tfm, total_steps, lr=5e-4):
        super().__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.viz_tfm = viz_tfm
        self.lr = lr
        self.total_steps = total_steps
        self.train_accuracy = Accuracy(task="multiclass", num_classes=model.n_classes)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=model.n_classes)
        self.train_miou = MeanIoU(num_classes=13)
        self.val_miou = MeanIoU(num_classes=13)
        self.automatic_optimization = False

        self.step_idx = 0
        
        # for n, m in self.model.named_modules():
        #     assert not hasattr(m, 'name')
        #     m.name = n

    def forward(self, x):
        return self.model(x)
        
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=self.lr, eps=1e-5)
        lr_scheduler = OneCycleLR(optimizer, self.lr, self.total_steps)
        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
    
    def training_step(self, batch, batch_idx):
        stage = 'train'
        logger = self.logger.experiment

        step_01 = max(int(0.01 * self.total_steps), 1)
        step_25 = max(int(0.25 * self.total_steps), 1)

        metrics = {"accuracy": self.train_accuracy, "miou": self.train_miou}
        loss, pred, label, mask = self.step(batch, batch_idx, stage, metrics)
        
        if self.step_idx % step_01 == 0:
            log_activations(logger, self.step_idx, self.model, batch['frame'])
        if self.step_idx % step_25 == 0:
            log_imgs(pred, label, mask, self.viz_tfm, logger, stage, self.step_idx)
        self.manual_optimization(loss)
        self.step_idx += 1
    
    def on_train_epoch_end(self):
        self.log('train_acc_epoch', self.train_accuracy)

    def validation_step(self, batch, batch_idx):
        stage = 'val'
        logger = self.logger.experiment

        metrics = {"accuracy": self.val_accuracy, "miou": self.val_miou}
        _, pred, label, mask = self.step(batch, batch_idx, stage, metrics)
        if batch_idx == 0:
            log_imgs(pred, label, mask, self.viz_tfm, logger, stage, self.step_idx)
    
    def step(self, batch, batch_idx, stage, metrics):
        item = batch
        img = item['frame']
        label = item['label']
        mask = item['mask']

        label[~mask] = 0
        
        pred = self.model(img)
        
        loss = self.loss_fn(pred, label)
        loss = loss[mask]
        loss = loss.mean()
        
        pred_f = torch.permute(pred, (0, 2, 3, 1)) # N,C,H,W -> N,H,W,C
        pred_f = torch.flatten(pred_f, 0, -2)      # N,H,W,C -> N*H*W,C
        mask_f = torch.flatten(mask)               # N,H,W   -> N*H*W
        pred_m = pred_f[mask_f, :]
        label_m = label[mask]
        metrics["accuracy"](pred_m, label_m)
        
        pred_labels = torch.argmax(pred, dim=1)
        mask_miou = (label != 0)
        pred_labels[~mask] = 0
        metrics["miou"](pred_labels, label)

        self.log(f"{stage}_acc_step", metrics["accuracy"])
        self.log(f"{stage}_mean_iou_step", metrics["miou"])
        self.log(f"{stage}_loss_step", loss.log10())

        return loss, pred, label, mask
    
    def manual_optimization(self, loss):
        optimizer = self.optimizers()
        optimizer.zero_grad()
        self.manual_backward(loss)
        
        p_old = {}
        for n, p in self.model.named_parameters():
            p_old[n] = p.detach().clone()
        
        optimizer.step()
        
        for n, p in self.model.named_parameters():
            optim_step = p.detach() - p_old[n]
            
            #log_rate = optim_step.abs().log1p() - p_old[n].abs().log1p()
            #log_rate_hist = np.histogram(log_rate.cpu().view(-1), 100)
            #self.logger.experiment.log({"log_update_rate_hist/" + str(n): wandb.Histogram(np_histogram=log_rate_hist)}, step=self.step_idx)
            
            self.logger.experiment.log({"ud/" + str(n): ((optim_step.std())/(p_old[n].std() + 1e-5)).log10()}, step=self.step_idx)
        
        lr_scheduler = self.lr_schedulers()
        lr_scheduler.step()

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()